In [1]:
import pandas as pd
import os, sys
import numpy as np

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../..')))

from utils.prediction_utils import *
""
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import make_scorer, r2_score

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler

random_seed = 42

In [2]:
base_dir = "/Users/andrew/Desktop/Harvard/idreos-research/gpu_profiling"
X, y = get_data("conv2d", base_dir, sample_rate=1.0)
X.info()

/Users/andrew/Desktop/Harvard/idreos-research/gpu_profiling/utils/time_utils.py:216: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dfs = pd.concat(dfs, axis=0, ignore_index=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 685125 entries, 0 to 685124
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   b             685125 non-null  int64  
 1   in_channels   685125 non-null  int64  
 2   iH            685125 non-null  int64  
 3   iW            685125 non-null  int64  
 4   out_channels  685125 non-null  int64  
 5   groups        685125 non-null  int64  
 6   kH            685125 non-null  int64  
 7   kW            685125 non-null  int64  
 8   stride        685125 non-null  int64  
 9   dilation      685125 non-null  int64  
 10  gflops        685125 non-null  float64
 11  dtype_16      685125 non-null  bool   
 12  dtype_32      685125 non-null  bool   
 13  dtype_b16     685125 non-null  bool   
 14  transposed_0  685125 non-null  bool   
 15  transposed_1  685125 non-null  bool   
dtypes: bool(5), float64(1), int64(10)
memory usage: 60.8 MB


In [3]:
df = pd.concat([X, y], axis=1)
df = df.query("time > 0").dropna()
X, y = df.drop(["time"], axis=1), df["time"]

In [4]:
X_train, X_val, X_test, y_train, y_val, y_test = get_train_test_split(X, y, return_concat=False)
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 444058 entries, 178626 to 121958
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   b             444058 non-null  int64  
 1   in_channels   444058 non-null  int64  
 2   iH            444058 non-null  int64  
 3   iW            444058 non-null  int64  
 4   out_channels  444058 non-null  int64  
 5   groups        444058 non-null  int64  
 6   kH            444058 non-null  int64  
 7   kW            444058 non-null  int64  
 8   stride        444058 non-null  int64  
 9   dilation      444058 non-null  int64  
 10  gflops        444058 non-null  float64
 11  dtype_16      444058 non-null  bool   
 12  dtype_32      444058 non-null  bool   
 13  dtype_b16     444058 non-null  bool   
 14  transposed_0  444058 non-null  bool   
 15  transposed_1  444058 non-null  bool   
dtypes: bool(5), float64(1), int64(10)
memory usage: 42.8 MB


# Neural Network Training

In [5]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [14]:
# X_train_scaled.shape
# X_val_scaled.shape
# X_test_scaled.shape

(95155,)

In [15]:
try:
    # X_train_t = torch.tensor(X_train_scaled, dtype=torch.float32)
    # y_train_t = torch.tensor(y_train.values, dtype=torch.float32)
    # X_val_t = torch.tensor(X_val_scaled, dtype=torch.float32)
    y_val_t = torch.tensor(y_val.values, dtype=torch.float32)
except Exception as e:
    print(f"Error occurred: {e}")

: 

In [ ]:

# Creating datasets
train_dataset = TensorDataset(X_train_t, y_train_t)
val_dataset = TensorDataset(X_val_t, y_val_t)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Define the Neural Network Architecture
class RegressionNN(nn.Module):
    def __init__(self):
        super(RegressionNN, self).__init__()
        self.fc1 = nn.Linear(X_train_t.shape[1], 50)
        self.fc2 = nn.Linear(50, 20)
        self.fc3 = nn.Linear(20, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Instantiate the model, loss function, and optimizer
model = RegressionNN()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets.view(-1, 1))
        loss.backward()
        optimizer.step()
    
    # Validation loop
    model.eval()
    with torch.no_grad():
        val_loss = 0
        for inputs, targets in val_loader:
            outputs = model(inputs)
            val_loss += criterion(outputs, targets.view(-1, 1)).item()
        val_loss /= len(val_loader)
    print(f'Epoch {epoch+1}, Validation Loss: {val_loss:.4f}')

# Evaluation on test set can be added similarly